# GERENCIAL ALFSTS/DIREP/EQPEM

In [1]:
import calendar
import os
import pandas as pd
import sys
from datetime import datetime
from sqlalchemy.sql import extract
from collections import defaultdict, OrderedDict
from sqlalchemy import create_engine
caminho_commons = os.path.join('..', '..', '..', 'ajna_docs', 'commons')
caminho_virasana = os.path.join('..', '..', '..', 'virasana')
sys.path.append(caminho_commons)
sys.path.append('..')
sys.path.append(caminho_virasana)
from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from bhadrasana.models import engine, db_session
from bhadrasana.models import Usuario
from bhadrasana.models.ovr import OVR, EventoOVR
from bhadrasana.models.rvfmanager import lista_rvfovr
from IPython.display import HTML

2023-06-01 17:21:40,850 ajna         INFO     Configuração de log efetuada


Fazendo log de erros e alertas no arquivo  ..\..\..\ajna_docs\commons\ajna_commons\flask\error.log
Fazendo log de atividade no arquivo  ..\..\..\ajna_docs\commons\ajna_commons\flask\access.log


C:\Users\25052288840\Anaconda3\lib\site-packages\requests\__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


# Ações de Vigilância por mês

In [2]:
def get_fichas_vigilancia():
    lista_ovrs = db_session.query(OVR).join(EventoOVR)\
                                      .filter(OVR.setor_id.in_((1, 2, 3)))\
                                      .filter(OVR.tipooperacao.in_((7, )))\
                                      .filter(EventoOVR.fase >= 3)\
                                      .order_by(OVR.id, EventoOVR.create_date).all()
    lista_fichas = set()
    for ovr in lista_ovrs: # Não inclui OVR duas vezes
        lista_fichas.add(ovr)
    return lista_fichas

In [8]:
def get_peso_apreensoes(session, rvfs):
    peso = 0.
    for rvf in rvfs:
        for apreensao in rvf.apreensoes:
            try:
                peso += float(apreensao.peso)
            except TypeError:
                pass
    return peso

def get_valor_tgs(ovr):
    valor = 0.
    for tg in ovr.tgs:
        try:
            valor += float(tg.valor)
        except TypeError:
            pass
    return valor

def preenche_linha_operacao(operacao):
    linha = OrderedDict()
    linha['Ficha'] = operacao.id
    linha['Equipe'] = operacao.setor.nome
    recinto = 'Não informado'
    if operacao.recinto:
        recinto = operacao.recinto.nome
    linha['Local da Operação'] = recinto
    linha['Observações'] = operacao.observacoes
    rvfs = lista_rvfovr(db_session, operacao.id)
    achados = []
    for rvf in rvfs:
        if rvf.descricao:
            achados.append(rvf.descricao)
    linha['Achados'] = ' -*- '.join(achados)
    linha['Supervisor'] = str(operacao.cpfauditorresponsavel)
    linha['Responsável'] = str(operacao.responsavel_cpf)
    valortgs = get_valor_tgs(operacao)
    pesoapreensoes = get_peso_apreensoes(db_session, rvfs)
    servidores_set = set()
    for evento in operacao.historico:
        servidores_set.add(evento.user_name)
        if evento.fase >=3:
            data_encerramento = evento.create_date
    linha['Data Encerramento'] = data_encerramento
    flags =  ' '.join([flag.nome for flag in operacao.flags])
    linha['K9'] = 'K9' in flags
    linha['Lancha'] = 'Lancha' in flags
    linha['Drone'] = 'Drone' in flags
    servidores = []
    for cpf in servidores_set:
        servidor = db_session.query(Usuario).filter(Usuario.cpf == cpf).one()
        servidores.append({'cpf': servidor.cpf, 'nome': servidor.nome})
    linha['Servidores'] = servidores
    linha['Perdimento'] = valortgs
    linha['Apreensao'] = pesoapreensoes
    return linha


In [9]:
lista_fichas = get_fichas_vigilancia()
lista_final = []
for ficha in lista_fichas:
    linha_operacao = preenche_linha_operacao(ficha)
    lista_final.append(linha_operacao)
df = pd.DataFrame(lista_final)
df = df.sort_values('Data Encerramento')

In [20]:
df['AnoMes'] = df.apply(lambda x: str(x['Data Encerramento'].year) + str(x['Data Encerramento'].month).zfill(2), axis=1)

In [21]:
df_anomes = df.groupby(df.AnoMes).agg(
    quantidade=pd.NamedAgg(column='Ficha', aggfunc='count'),
    perdimentos=pd.NamedAgg(column='Perdimento', aggfunc='sum'),
    apreensoes=pd.NamedAgg(column='Apreensao', aggfunc='sum'),
    K9=pd.NamedAgg(column='K9', aggfunc='sum'),
    Lancha=pd.NamedAgg(column='Lancha', aggfunc='sum'),
    Drone=pd.NamedAgg(column='Drone', aggfunc='sum'),
).reset_index()
df_anomes

,index,quantidade,perdimentos,apreensoes,K9,Lancha,Drone
0,202011,1,0.0,0.0,0,0,0
1,202112,1,0.0,0.0,0,0,0
2,20212,1,0.0,0.0,0,0,0
3,20213,1,0.0,0.0,0,0,0
4,20214,1,0.0,0.0,0,0,0
5,202210,1,0.0,0.0,0,0,0
6,202212,1,0.0,0.0,0,0,0
7,20225,1,0.0,0.0,0,0,0
8,20226,2,0.0,0.0,0,0,0
9,20227,2,0.0,0.0,0,0,0


In [24]:
HTML(df.drop(columns=['AnoMes',]).to_html(index=False))

Ficha,Equipe,Local da Operação,Observações,Achados,Supervisor,Responsável,Data Encerramento,K9,Lancha,Drone,Servidores,Perdimento,Apreensao
3272,ALFSTS/DIREP,EMBRAPORT - EMPRESA BRASILEIRA DE TERMINAIS PORTUA,Ficha para controle de qualidade e auditoria dos equipamentos de escaneamento,,25052288840,25052288840,2023-05-08 11:07:56,False,False,False,"[{'cpf': '25052288840', 'nome': 'Ivan da Silva Brasílico'}]",0.0,0.0
6148,ALFSTS/DIREP/EQREXP,ALFANDEGA,"Quarta-feira dia 10/05\r\n\r\nRonda pelos Recintos Alfandegados:\r\n\r\nTransbrasa, Santos Brasil Logística, Deicmar, Bandeirantes, Marimex e Eudmarco.\r\n\r\n\r\nServidores - Márcio e Alysson\r\n\r\n","Transbrasa:\r\n\r\nQuatro balanças (duas de entrada e duas de saída);\r\nCaminhão que entrega um container e retira outro deve fazer a saída física e nova entrada para retirar o outro container.\r\n\r\nSantos Brasil Logística (Santos):\r\nUma balança na entrada e uma na saída;\r\nTem balança interna, mas não aceitam que o caminhão retire um caminhão na mesma entrada da entrega de outro container.\r\n\r\nBandeirantes Deicmar Clia:\r\nQuatro balanças no gate, que funcionam nos dois sentidos, de acordo com o maior fluxo;\r\nNos casos de entrada para entrega e retirada de container, o caminhão é pesado vazio no gate. Se for possível, retorna de ré, se não for possível, faz um balão na entrada do recinto e entra para carregar a unidade de saída.\r\n\r\nBandeirantes:\r\nDuas balanças, uma de entrada e uma de saída.\r\nSão raros os casos de entrega e retirada na mesma entrada, mas quando acontece, o caminhão não é pesado vazio.\r\n\r\nMarimex:\r\nTrês balanças de entrada.\r\nNão tem balança interna.\r\nDuas balanças na saída.\r\nNo caso de entrega e retida de container na mesma entrada, o caminhão é pesado vazio. Se for da frota interna, manobra e carrega o container da saída. Se for caminhão externo, tem que sair e dar a volta e entrar novamente.\r\n\r\nEudmarco:\r\nUma balança na entrada e uma balança na saída.\r\nNo caso da entrada para entrega e retirada de container na mesma entrada, recebemos duas respostas diferentes:\r\n1 - Caminhão descarrega o container, pesa vazio na balança de saída, dá ré e carrega o outro.\r\n2 - Caminhão tem que fazer a saída física, dar a volta no canal e retorna para retirar o outro container.\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n",None,None,2023-05-11 14:29:17,False,False,False,"[{'cpf': '00579600629', 'nome': 'Alysson Ribeiro Lopes'}]",0.0,0.0
3575,ALFSTS/DIREP,BANDEIRANTES DEICMAR - CLIA ALEMOA,None,,25052288840,25052288840,2023-05-08 11:13:20,False,False,False,"[{'cpf': '25052288840', 'nome': 'Ivan da Silva Brasílico'}]",0.0,0.0
6149,ALFSTS/DIREP/EQREXP,ALFANDEGA,"Quinta-feira 11/05\r\nRonda no canal do Porto.\r\nVisita da ABIN.\r\nTripulação: Sandro, Márcio e Tadeu",,None,None,2023-05-11 14:31:29,False,False,False,"[{'cpf': '00579600629', 'nome': 'Alysson Ribeiro Lopes'}]",0.0,0.0
3577,ALFSTS/DIREP,ECOPORTO SANTOS - OPERADOR TECONDI,None,,25052288840,25052288840,2023-05-08 13:23:07,False,False,False,"[{'cpf': '25052288840', 'nome': 'Ivan da Silva Brasílico'}]",0.0,0.0
3578,ALFSTS/DIREP,MARIMEX IPA,None,,25052288840,25052288840,2023-05-08 11:15:21,False,False,False,"[{'cpf': '25052288840', 'nome': 'Ivan da Silva Brasílico'}]",0.0,0.0
6156,ALFSTS/DIREP/EQREXP,ALFANDEGA,"Sexta-feira 12/05\r\nRonda no canal do porto e fundeadores.\r\n\r\nVisita da Aduana Americana\r\n\r\nOperação de mergulho em conjunto com Marinha e PF no navio Berge Daisets, atracado no Armazém 19. \r\nMergulho encerrado no Berge Daisets sem resultado.\r\n\r\nOperação de mergulho no navio Western Oslo, atracado ao lado do navio Berge Daisets.\r\nMergulho encerrado sem resultado.\r\n\r\nTripulação: Márcio, Alysson, Tioyama e Ivan.",,None,None,2023-05-12 18:15:59,False,False,False,"[{'cpf': '00579600629', 'nome': 'Alysson Ribeiro Lopes'}]",0.0,0.0
3582,ALFSTS/DIREP,SANTOS BRASIL LOGISTICA S/A - CLIA SANTOS,None,,25052288840,25052288840,2023-05-08 11:16:08,False,False,False,"[{'cpf': '2505